In [1]:
import pandas as pd
import numpy as np
import glob, time, os, cv2, csv

try:
    %env CUDA_DEVICE_ORDER=PCI_BUS_ID
    %env CUDA_VISIBLE_DEVICES=5
except NameError:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "5"

df_path = '/home/Erdal.Genc/covid_work/PadChest_csv/data_preprocessing_PadChest_PadChest_labels_ED_filtered.csv'
img_path = '/mnt/dsets/ChestXrays/PadChest/image_zips'


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16288283281908442334
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9602973470727666231
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 176422912
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11397539387719647330
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:14:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11540801806662282487
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
### Removing Some Chars in the Labels column
data_new=pd.read_csv(df_path, low_memory=False)

data_copy= data_new.copy()
data_copy['new_labels']=data_copy['new_labels'].str.replace("[","")
data_copy['new_labels']=data_copy['new_labels'].str.replace("]","")
data_copy['new_labels']=data_copy['new_labels'].str.replace("'","")

print(len(data_copy))


102893


In [3]:
### loading each Image and arranging pixel intensity
### Load this if img and corrupted files were not saved
import time  ### this cells takes a lot of time to process
start = time.time() 
##Creating new column for path of each image
data_copy['Path'] =  'path'

CorruptedFiles = []

for root, dirnames, filename in os.walk(img_path):
    for i in range(len(data_copy['ImageID'])):
        if data_copy['ImageID'][i] in filename:
            try:  ###to prevent FileNotFound Error
                try: ###to prevent Broken PNG file Error
                    ## We need to load each image to if there is a broken PNG or missing PNG
                    #img = image.load_img(os.path.join(str(root),data_copy['ImageID'][i]), target_size=(224,224))
                    data_copy['Path'][i] = os.path.join(str(root),data_copy['ImageID'][i])
                    i+=1
                except SyntaxError:
                    print("Broken PNG: Deleting from CSV!!!", data_copy['ImageID'][i])
                    CorruptedFiles.append(data_copy['ImageID'][i])
            except OSError:
                print( "FileNotFound: Deleting from CSV!!!", data_copy['ImageID'][i])
                CorruptedFiles.append(data_copy['ImageID'][i])
        else:
            continue

end = time.time()
print("Time Elapsed:", end-start)

/home/Erdal.Genc/anaconda3/envs/conda-env2/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Time Elapsed: 421.84553575515747


In [4]:
## Deleting corrupted files
Corr_files = open('/mnt/home/Erdal.Genc/corrupted_images.txt' , 'r')
for ele in Corr_files.readlines():
    ele = ele.replace("\n", "")
    data_copy = data_copy[data_copy['ImageID'] != ele]

print('Missing Files Deleted. New Length: ', len(data_copy['ImageID']))

Missing Files Deleted. New Length:  102874


In [6]:
##Saving df file
data_copy.to_csv('/home/Erdal.Genc/covid_work/PadChest_csv/Padchest_filtered_withPath.csv', index =False)